### 예스24 크롤링


In [44]:
#webdriver-manager 라이버러리 불러오기
from webdriver_manager.chrome import ChromeDriverManager
ChromeDriverManager().install()


'/Users/baggiung/.wdm/drivers/chromedriver/mac64/124.0.6367.60/chromedriver-mac-x64/chromedriver'

In [62]:
from selenium import webdriver
browser = webdriver.Chrome()

In [63]:
url= 'https://www.google.com/'
browser.get(url)

In [47]:
from selenium.webdriver.common.by import By
 
browser.find_element(By.CLASS_NAME, 'uU7dJb').textadsa

AttributeError: 'WebElement' object has no attribute 'textadsa'

In [64]:
url= 'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=1&pageSize=24'
browser.get(url)

In [ ]:
#1페이지의 링크 데이터 전부 수집

### 한 개의 베스트 셀러 링크 수집, class 이름 = gd_name
browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href')

### 1페이지 전체의 링크 데이터
#browser.find_element(By.CLASS_NAME, 'gd_name') # element: 요소
datas = browser.find_elements(By.CLASS_NAME, 'gd_name') # element: 리스트

for i in datas:
    print(i.get_attribute('href'))



In [74]:
#3페이지까지의 링크 데이터 전부 수집
# https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=3&pageSize=24


import time
link_list = []
for i in range(1, 4):
    print("*"*10, f'현재 {i} 페이지 수집 중 입니다.', '*'*10)

    url = f'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber={i}&pageSize=24'
    browser.get(url)

    browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href')
    datas = browser.find_elements(By.CLASS_NAME, 'gd_name') # element: 리스트

    for i in datas:
        link = i.get_attribute('href')
        link_list.append(link)

    time.sleep(3)

print(link_list)

********** 현재 1 페이지 수집 중 입니다. **********
********** 현재 2 페이지 수집 중 입니다. **********
********** 현재 3 페이지 수집 중 입니다. **********
['https://www.yes24.com/Product/Goods/124999476', 'https://www.yes24.com/Product/Goods/126010792', 'https://www.yes24.com/Product/Goods/125992096', 'https://www.yes24.com/Product/Goods/125295101', 'https://www.yes24.com/Product/Goods/126029503', 'https://www.yes24.com/Product/Goods/125491624', 'https://www.yes24.com/Product/Goods/125651056', 'https://www.yes24.com/Product/Goods/125970283', 'https://www.yes24.com/Product/Goods/125576919', 'https://www.yes24.com/Product/Goods/117014613', 'https://www.yes24.com/Product/Goods/122120495', 'https://www.yes24.com/Product/Goods/123675187', 'https://www.yes24.com/Product/Goods/126029190', 'https://www.yes24.com/Product/Goods/125973831', 'https://www.yes24.com/Product/Goods/90926731', 'https://www.yes24.com/Product/Goods/96547408', 'https://www.yes24.com/Product/Goods/123400303', 'https://www.yes24.com/Product/Goods/12404381

In [ ]:
len(link_list)

72

In [ ]:
from selenium.webdriver.common.by import By
url= 'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=1&pageSize=24'
browser.get(url)

In [73]:
# for link in link_list:
#     # 상세 페이지로 이동
#     browser.get(link)

browser.get(link_list[0])

title = browser.find_element(By.CLASS_NAME, 'gd_name').text
author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text
publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
rating = browser.find_element(By.CLASS_NAME, 'yes_b').text
reviews = browser.find_element(By.CLASS_NAME, 'txC_blue').text

sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]

price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]
rangking = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[0]
rangking_weeks= browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[1]
#rangking = browser.find_element(By.CLASS_NAME, 'gd_best').find_element(By.TAG_NAME, 'a').text
#rangking2 = browser.find_element(By.CLASS_NAME, 'gd_best').find_element(By.TAG_NAME, 'dd').text






In [67]:
text = '| 판매지수 485,568'

text.split(" ")[2]

'485,568'

In [50]:
text = '베스트 국내도서 1위 | 국내도서 1위 5주'

text.split(" | ")[0].split(" ")[2]

'1위'

## 33강~36강 코드


In [138]:
# 데이터 베이스 연동 후 -> 수집한 데이터를 DB에 저장 (csv)
# 상세 페이지 이동 후 데이터 크롤링
import pymysql # pip install pymysql
import time
import re
from datetime import datetime
from selenium.common.exceptions import NoSuchElementException


conn = pymysql.connect(
    host='localhost',
    user='root',
    password='!rldnd12',
    db='yse24',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

with conn.cursor() as cur:

    for link in link_list:
        browser.get(link)

        title = browser.find_element(By.CLASS_NAME, 'gd_name').text
        author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
        publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text

        # 2023년 12월 25일 -> 2023-12-24
        publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text

        match = re.search(r'(\d+)년 (\d+)월 (\d+)일', publishing)

        if match:
            year, month, day = match.groups()
            data_obj = datetime(int(year), int(month), int(day)) 
            publishing = data_obj.strftime("%Y-%m-%d")
        else:
            publishing = "2023-01-01"

    

        try:
            rating_element = browser.find_element(By.CLASS_NAME, "yes_b")
            rating_text = rating_element.text
    
            rating = float(rating_text)
        
        except ValueError:
            # 변환할 수 없는 경우 0으로 처리
            rating = 0
        except NoSuchElementException:
            rating = 0


        # try:
        #     review = browser.find_element(By.CLASS_NAME, 'txC_blue').text
        #     review == int(review.replace(",", ""))
        # except NoSuchElementException:
        #     review = 0
        # except (ValueError, TypeError):
        #     review = 0
            

        try:
            review_element = browser.find_element(By.CLASS_NAME, "txC_blue")
            review = int(review_element.text.replace(",", ""))
        except NoSuchElementException:
            review = 0
        except (ValueError, TypeError):
            review = 0
        


        sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
        sales = int(sales.replace(",", ""))

        price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]
        price = int(price.replace(",", ""))

        full_text = browser.find_element(By.CLASS_NAME, 'gd_best').text
        parts = full_text.split(" | ")
        
        print('full_text : ', full_text) #베스트만 출력

        # if len(parts) == 1:
        #     ranking = 0
        #     ranking_weeks = 0
            
        # else: 

        try: 
                ranking_part = parts[0]
                ranking = ''.join(filter(str.isdigit, ranking_part)) # 숫자만 추출
        except:
                ranking = 0
            
        try:    
                # 국내도서 top20 3주
                ranking_weeks_part = parts[1]
                ranking_weeks = ''.join(filter(str.isdigit, ranking_weeks_part.split()[-1]))

        except:
                rangking_weeks = 0

        sql = """
        INSERT INTO Books(title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks)
        VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """

        cur.execute(sql, (title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks))
        conn.commit()

        time.sleep(2)


full_text :  베스트 국내도서 1위 | 국내도서 1위 6주
full_text :  베스트 국내도서 2위 | 국내도서 top20 1주
full_text :  베스트 국내도서 3위 | 국내도서 top20 1주
full_text :  베스트 국내도서 4위 | 국내도서 top20 5주
full_text :  베스트 국내도서 5위 | 국내도서 top20 1주
full_text :  베스트 국내도서 6위 | 국내도서 top20 5주
full_text :  베스트 국내도서 7위 | 국내도서 top20 1주
full_text :  베스트 국내도서 8위 | 국내도서 top20 1주
full_text :  베스트 국내도서 9위 | 국내도서 top20 3주
full_text :  베스트 국내도서 10위 | 국내도서 1위 22주
full_text :  베스트 국내도서 11위 | 국내도서 1위 12주
full_text :  베스트 국내도서 12위 | 국내도서 top20 17주
full_text :  베스트 국내도서 13위 | 국내도서 top20 1주
full_text :  베스트 국내도서 14위 | 국내도서 top20 1주
full_text :  베스트 국내도서 15위 | 국내도서 top20 4주
full_text :  베스트 국내도서 16위 | 국내도서 top20 21주
full_text :  베스트 국내도서 17위 | 국내도서 top20 15주
full_text :  베스트 국내도서 18위 | 국내도서 top20 18주
full_text :  베스트 국내도서 19위 | 국내도서 top20 6주
full_text :  베스트 국내도서 20위 | 국내도서 top20 4주
full_text :  베스트 국내도서 21위 | 국내도서 top20 2주
full_text :  베스트 국내도서 22위 | 국내도서 top100 1주
full_text :  베스트 국내도서 23위 | 국내도서 top100 3주
full_text :  베스트 국내도서 24위 | 국내도서 top20 2주
fu